In [ ]:
""" Analyze remote sensing data to estimate when irrigation ceases """


In [6]:
""" Get the images from Earth Engine """
import ee
ee.Initialize()

# local imports
from ee_api.ee_utils import landsat_masked

IRR = 'projects/ee-dgketchum/assets/IrrMapper/IrrMapperComp'
BASINS = 'users/dgketchum/gages/gage_basins'

year = 2022
debug = False
bucket = 'wudr'
basin = '12334550' 

# download images clipped to the basin of interes
basin = ee.FeatureCollection(BASINS).filterMetadata('STAID', 'equals', basin)
s, e = '1987-01-01', '2021-12-31'

# get irrmapper
irr_coll = ee.ImageCollection(IRR)
coll = irr_coll.filterDate(s, e).select('classification')
remap = coll.map(lambda img: img.lt(1))
irr_min_yr_mask = remap.sum().gte(5)
irr = irr_coll.filterDate('{}-01-01'.format(year),
                          '{}-12-31'.format(year)).select('classification').mosaic()
irr_mask = irr_min_yr_mask.updateMask(irr.lt(1))

# get landsat spectral (NDVI) information
# we used to use ETof, but it was not sensitive enough
coll = landsat_masked(year, basin).map(lambda x: x.normalizedDifference(['B5', 'B4']))
scenes = coll.aggregate_histogram('system:index').getInfo()

for img_id in scenes:
    
    # several landsat footprints will be detected, 040028 is the most centered over UCF
    if '040028' in img_id:
        pass
    else:
        continue

    img = coll.filterMetadata('system:index', 'equals', img_id).first()
    img = img.clip(basin.geometry()).mask(irr_mask).multiply(1000).int()

    if debug:
        point = ee.Geometry.Point([-112.75608, 46.31405])
        data = img.sample(point, 30).getInfo()
        print(data['features'])

    task = ee.batch.Export.image.toCloudStorage(
        img,
        description='NDVI_{}'.format(img_id),
        bucket=bucket,
        region=basin.geometry(),
        crs='EPSG:5070',
        scale=30)

    task.start()
    print(img_id)


1_1_1_1_LE07_040028_20220131
1_1_1_1_LE07_040028_20220304
1_1_1_1_LE07_040028_20220320
1_1_1_1_LE07_040028_20220405
1_1_1_1_LE07_040028_20220513
1_1_1_1_LE07_040028_20220616
1_1_1_1_LE07_040028_20220703
1_1_1_1_LE07_040028_20220720
1_1_1_1_LE07_040028_20220806
1_1_1_1_LE07_040028_20220823
1_1_1_1_LE07_040028_20220909
1_1_1_1_LE07_040028_20220926
1_1_1_1_LE07_040028_20221013
1_1_1_1_LE07_040028_20221030
1_1_1_1_LE07_040028_20221116
1_1_1_2_LC08_040028_20220123
1_1_1_2_LC08_040028_20220208
1_1_1_2_LC08_040028_20220224
1_1_1_2_LC08_040028_20220312
1_1_1_2_LC08_040028_20220328
1_1_1_2_LC08_040028_20220413


KeyboardInterrupt: 

In [ ]:
""" Use gsutil command line tool to transfer the images from the google cloud storage bucket to local machine """
# see: https://cloud.google.com/storage/docs/gsutil
# something like
# gsutil -m mv gs://wudr/NDVI_*.tif /home/username/Downloads/12334550/landsat/input

In [3]:
# some helper functions for the following algorithm implementation

def get_list_info(tif_dir, year, list_=False):
    """ Pass list in place of tif_dir optionally """
    if list_:
        l = tif_dir
    else:
        l = [os.path.join(tif_dir, x) for x in os.listdir(tif_dir) if
             x.endswith('.tif') and '_{}'.format(year) in x]
    srt = sorted([x for x in l], key=lambda x: int(x.split('.')[0][-4:]))
    d = [x.split('.')[0][-8:] for x in srt]
    d_numeric = [int(x) for x in d]
    dstr = ['{}-{}-{}'.format(x[:4], x[4:6], x[-2:]) for x in d]
    dates_ = [pd.to_datetime(x) for x in dstr]
    doy = [int(dt.strftime('%j')) for dt in dates_]
    return l, d_numeric, doy


def read_rasters(file_list, idx):
    da = None
    first = True
    for tif in file_list:
        if first:
            da, w = read_raster(tif, idx, return_win=True)
            first = False
        else:
            d = read_raster(tif, idx)
            da = np.append(da, d, axis=0)
    return da, w


def get_doy_index(d, idx):
    broadcast = np.broadcast_to(np.array(idx), np.ones_like(d).T.shape).T
    seq = broadcast * np.ones_like(d)
    doy_idx = np.where(d > 50, seq, 0)
    doy_idx = doy_idx.max(axis=0).astype(np.uint16)
    return doy_idx


def read_raster(t, c, return_win=False):
    win = Window(row_off=c[0], col_off=c[1], height=c[2], width=c[3])
    with rasterio.open(t, 'r') as src:
        a = src.read(1, window=win)
        window = src.window_transform(win)
    a = a[np.newaxis, :, :]
    if return_win:
        return a, window
    return a


def interp(a, over='nan'):
    def pad(data):
        if np.count_nonzero(data) == 0:
            return data
        if over == 'nan':
            good = np.isfinite(data)[0]
        else:
            good = np.nonzero(data)[0]
        x = np.arange(data.shape[0])
        fp = data[good]
        _interp = np.interp(x, good, fp)
        return _interp

    a = np.apply_along_axis(pad, 0, a)
    return a

In [25]:
# set-up to run the algorithm, modify paths as needed
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import rasterio
from rasterio.merge import merge
from rasterio.windows import Window

root = '/media/research/IrrigationGIS/Montana/water_rights'
if not os.path.exists(root):
    root = '/home/dgketchum/data/IrrigationGIS/Montana/water_rights'

basin_ = '12334550'
tif_dir = os.path.join(root, 'landsat', 'ndvi', basin_, 'input')
out_tif = os.path.join(root, 'landsat', 'ndvi', basin_, 'merged')
year = 2016
overwrite = False
chunk = 1000
threshold = 600

In [ ]:
""" The irrigation date-of-cessation algorithm. """


file_list, int_dates, doy = get_list_info(tif_dir, year, list_=False)

file_name = os.path.join(out_tif, '{}.tif'.format(year))
if os.path.exists(file_name) and not overwrite:
    print('{} exists already'.format(file_name))

with rasterio.open(file_list[0], 'r') as src:
    meta = src.meta

h, w = meta['height'], meta['width']

t = [(x, y, chunk, chunk) for x in range(0, h, chunk) for y in range(0, w, chunk)]
tiffs, doy_tiffs = [], []

dtstr_keys = {i: k for i, k in enumerate(int_dates)}
doy_keys = {i: k for i, k in enumerate(doy)}

for elem, c in tqdm(enumerate(t), total=len(t)):

    file_name = os.path.join(out_tif, '{}_{}.tif'.format(year, elem))
    doy_pth = file_name.replace('.tif', '_doy.tif')
    file_check = [os.path.exists(file_name), os.path.exists(doy_pth), not overwrite]
    if all(file_check):
        print('{} and {} exist, skipping'.format(os.path.basename(file_name),
                                                 os.path.basename(doy_pth)))
        continue

    # peak-finding algorithm
    arr, w = read_rasters(file_list, c)
    arr = arr.astype(float)
    arr[arr == 0] = np.nan
    max_, mean_ = np.nanmax(arr, axis=0), np.nanmean(arr, axis=0)
    ct = np.count_nonzero(arr, axis=0)
    arr[0] = np.where(np.isnan(arr[0]), np.nanmin(arr, axis=0), arr[0])
    arr[np.isnan(arr)] = 0
    arr = interp(arr, over='zero')
    da = np.diff(arr, axis=0)
    zero = np.zeros((da.shape[0] + 1, da.shape[1], da.shape[2]))
    thresh = np.ones_like(arr) * threshold
    forward, back = np.append(da, zero[0:1], axis=0), np.append(zero[0:1], da, axis=0)
    peaks = ((back > zero) & (forward < zero) & (arr > thresh))
    peaks = peaks.flatten()
    idx = np.broadcast_to(np.arange(arr.shape[0]), np.ones_like(arr).T.shape).T.flatten()
    idx[~peaks] = 0
    idx = idx.reshape(arr.shape).max(axis=0)
    dtstr = np.vectorize(dtstr_keys.get)(idx)
    dtstr[idx == 0] = 0
    doy = np.vectorize(doy_keys.get)(idx)
    doy[idx == 0] = 0

    if np.count_nonzero(arr) == 0:
        continue

    meta['height'] = c[2]
    meta['width'] = c[3]
    meta['count'] = 5
    meta['transform'] = w
    meta['dtype'] = 'uint32'
    meta['nodata'] = 0

    description = ['date', 'doy', 'count', 'max', 'mean']
    data = [dtstr, doy, ct, max_, mean_]
    with rasterio.open(file_name, 'w', **meta) as dst:
        for i, (dat, desc) in enumerate(zip(data, description)):
            dst.write(dat, indexes=i + 1)
            dst.set_band_description(i + 1, desc)

    tiffs.append(file_name)

t_src = []
for t in tiffs:
    ts = rasterio.open(t)
    t_src.append(ts)

mosaic_t, out_trans_t = merge(t_src)

out_meta_t = ts.meta.copy()

out_meta_t.update({'height': mosaic_t.shape[1],
                   'width': mosaic_t.shape[2],
                   'transform': out_trans_t})

out_t = os.path.join(out_tif, '{}.tif'.format(year))
with rasterio.open(out_t, 'w', **out_meta_t) as dst:
    dst.write(mosaic_t)

[os.remove(x) for x in tiffs]

/media/research/IrrigationGIS/Montana/water_rights/landsat/ndvi/12334550/merged/2016.tif exists already


  0%|                                                                                                                                                                                       | 0/25 [00:00<?, ?it/s]/home/dgketchum/miniconda3/envs/opnt/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: All-NaN slice encountered
/home/dgketchum/miniconda3/envs/opnt/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: Mean of empty slice
/home/dgketchum/miniconda3/envs/opnt/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: All-NaN slice encountered
 52%|██████████████████████████████████████████████████████████████████████████████████████████▍                                                                                   | 13/25 [01:03<01:00,  5.04s/it]

In [7]:
# plot data

from rasterio.plot import show

import matplotlib.pyplot as plt

src = rasterio.open(out_t)

show(src, cmap='viridis')
plt.show()

NameError: name 'out_t' is not defined